# Purpose

The purpose of this notebook is to do some feature engineering and modelling on our flight data that I retrieved from the Bureau of Transportation Statistics. Using this data, I will determine which models to use to predict whether a flight will be delayed or not. 

First, I will import the libraries that I will be using as well as the datasets that I will be using.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline
sns.set(style='whitegrid', palette='pastel', font_scale=1.25)

In [2]:
df = pd.read_csv('cleaned_airline_data_jan.csv')
df.head()

,year,month,day_of_week,fl_date,op_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name,...,arr_time,arr_delay,arr_delay_new,actual_elapsed_time,flights,distance,dep_hour,arr_hour,Code,Description
0,2019,1,6,2019-01-19,9E,N176PQ,MSP,"Minneapolis, MN",CVG,"Cincinnati, OH",...,1832.0,-25.0,0.0,106.0,1.0,596.0,15,18,9E,Endeavor Air Inc.
1,2019,1,7,2019-01-20,9E,N309PQ,MSP,"Minneapolis, MN",CVG,"Cincinnati, OH",...,1825.0,-37.0,0.0,93.0,1.0,596.0,15,18,9E,Endeavor Air Inc.
2,2019,1,1,2019-01-21,9E,N176PQ,MSP,"Minneapolis, MN",CVG,"Cincinnati, OH",...,1845.0,-17.0,0.0,118.0,1.0,596.0,15,18,9E,Endeavor Air Inc.
3,2019,1,2,2019-01-22,9E,N135EV,MSP,"Minneapolis, MN",CVG,"Cincinnati, OH",...,1839.0,-23.0,0.0,107.0,1.0,596.0,15,18,9E,Endeavor Air Inc.
4,2019,1,3,2019-01-23,9E,N294PQ,MSP,"Minneapolis, MN",CVG,"Cincinnati, OH",...,1850.0,-12.0,0.0,120.0,1.0,596.0,15,18,9E,Endeavor Air Inc.


In [3]:
df.columns

Index(['year', 'month', 'day_of_week', 'fl_date', 'op_unique_carrier',
       'tail_num', 'origin', 'origin_city_name', 'dest', 'dest_city_name',
       'crs_dep_time', 'dep_time', 'dep_delay', 'dep_delay_new',
       'crs_arr_time', 'arr_time', 'arr_delay', 'arr_delay_new',
       'actual_elapsed_time', 'flights', 'distance', 'dep_hour', 'arr_hour',
       'Code', 'Description'],
      dtype='object')

In [4]:
# Drop unnecessary columns
df.drop(['origin_city_name','dest_city_name', 'flights'], axis=1, inplace=True)

## Feature Engineering

Next, I will create two new columns. Based on the tail number, I will create a previous arrival delay column. The tail number of a plane is much like a car's license plate. As such, I can use this number to group the same plan together and see if specific planes have a tendency to be delayed without creating dummy variables for all 5000+ planes.

In [5]:
df['prev_arr_delay'] = (df.sort_values(['fl_date', 'arr_time'], ascending=True)
                      .groupby(['tail_num'], as_index=True)
                      .shift()['arr_delay_new'])

In [6]:
# Taking the above, create average previous delay by airplane
df['avg_prev_arr_delay'] = (df.groupby('tail_num').cumsum()['prev_arr_delay']/(df.groupby('tail_num').cumcount()+1))

In [7]:
df.replace({'prev_arr_delay': np.nan}, 0, inplace=True)
df.replace({'avg_prev_arr_delay': {np.nan: 0, np.inf: 0}}, inplace=True)
df.head()

,year,month,day_of_week,fl_date,op_unique_carrier,tail_num,origin,dest,crs_dep_time,dep_time,...,arr_delay,arr_delay_new,actual_elapsed_time,distance,dep_hour,arr_hour,Code,Description,prev_arr_delay,avg_prev_arr_delay
0,2019,1,6,2019-01-19,9E,N176PQ,MSP,CVG,1556,1546.0,...,-25.0,0.0,106.0,596.0,15,18,9E,Endeavor Air Inc.,0.0,0.0
1,2019,1,7,2019-01-20,9E,N309PQ,MSP,CVG,1556,1552.0,...,-37.0,0.0,93.0,596.0,15,18,9E,Endeavor Air Inc.,0.0,0.0
2,2019,1,1,2019-01-21,9E,N176PQ,MSP,CVG,1556,1547.0,...,-17.0,0.0,118.0,596.0,15,18,9E,Endeavor Air Inc.,0.0,0.0
3,2019,1,2,2019-01-22,9E,N135EV,MSP,CVG,1556,1552.0,...,-23.0,0.0,107.0,596.0,15,18,9E,Endeavor Air Inc.,6.0,6.0
4,2019,1,3,2019-01-23,9E,N294PQ,MSP,CVG,1556,1550.0,...,-12.0,0.0,120.0,596.0,15,18,9E,Endeavor Air Inc.,0.0,0.0


In [8]:
top20 = pd.read_csv('top20_airports.csv')
top20

,airports,count,latitude,longitude
0,ATL,61580,33.6558,-84.4333
1,ORD,46598,41.9796,-87.8825
2,DFW,45428,32.9222,-97.0409
3,CLT,37553,35.2226,-80.9460
4,DEN,36452,39.8396,-104.6720
5,LAX,35315,33.9456,-118.3910
6,PHX,29092,33.4376,-112.0300
7,IAH,28890,29.9784,-95.3424
8,LGA,26694,40.7731,-73.8756
9,SFO,26141,37.6148,-122.3920


In [9]:
top20_airport = pd.read_csv('top20_airports.csv')['airports'].tolist()
df_sub = df[df['origin'].isin(top20_airport)]
df_sub = df_sub[df_sub['dest'].isin(top20_airport)]

In [10]:
df_sub['fl_date'] = pd.to_datetime(df_sub.fl_date, format='%Y-%m-%d')

In [11]:
df_sub.head()

,year,month,day_of_week,fl_date,op_unique_carrier,tail_num,origin,dest,crs_dep_time,dep_time,...,arr_delay,arr_delay_new,actual_elapsed_time,distance,dep_hour,arr_hour,Code,Description,prev_arr_delay,avg_prev_arr_delay
289,2019,1,7,2019-01-06,9E,N933XJ,EWR,MSP,1937,2117.0,...,103.0,103.0,196.0,1008.0,19,23,9E,Endeavor Air Inc.,99.0,61.333333
290,2019,1,1,2019-01-07,9E,N297PQ,EWR,MSP,1937,1932.0,...,-6.0,0.0,192.0,1008.0,19,21,9E,Endeavor Air Inc.,0.0,0.000000
291,2019,1,2,2019-01-08,9E,N298PQ,EWR,MSP,1937,1927.0,...,-20.0,0.0,183.0,1008.0,19,21,9E,Endeavor Air Inc.,0.0,0.000000
292,2019,1,3,2019-01-09,9E,N296PQ,EWR,MSP,1937,1945.0,...,13.0,13.0,198.0,1008.0,19,22,9E,Endeavor Air Inc.,9.0,4.500000
293,2019,1,4,2019-01-10,9E,N906XJ,EWR,MSP,1937,2116.0,...,75.0,75.0,169.0,1008.0,19,23,9E,Endeavor Air Inc.,97.0,97.000000


In [12]:
weather = pd.read_csv('2019_1_weather.csv', index_col=None)
weather.head()

,latitude,longitude,time,temperature_high,temperature_low,precip_intensity,precip_intensity_max,precip_intensity_maxtime,precip_type,wind_gust,wind_gust_time,wind_speed,wind_bearing,cloud_cover,visibility
0,33.6558,-84.4333,1546318800,64.12,51.19,0.0061,0.0405,1.546326e+09,rain,7.29,1546318800,1.77,310,0.77,2.49
1,33.6558,-84.4333,1546405200,57.07,49.10,0.0146,0.0764,1.546481e+09,rain,7.33,1546452000,1.29,28,0.84,2.02
2,33.6558,-84.4333,1546491600,54.52,53.34,0.0118,0.0798,1.546495e+09,rain,8.03,1546538400,1.26,7,1.00,1.89
3,33.6558,-84.4333,1546578000,64.76,43.52,0.0476,0.2179,1.546600e+09,rain,20.46,1546639200,1.54,201,0.91,2.68
4,33.6558,-84.4333,1546664400,58.52,41.86,0.0000,0.0000,NaN,NaN,15.32,1546714800,5.80,291,0.41,3.23


In [13]:
weather['datetime'] = pd.to_datetime(weather['time'], unit='s')

In [14]:
weather = weather.round(4)

In [15]:
weather.shape

(640, 16)

In [16]:
wa = pd.merge(weather, top20, how='left', on=['latitude', 'longitude'])
wa.head()

,latitude,longitude,time,temperature_high,temperature_low,precip_intensity,precip_intensity_max,precip_intensity_maxtime,precip_type,wind_gust,wind_gust_time,wind_speed,wind_bearing,cloud_cover,visibility,datetime,airports,count
0,33.6558,-84.4333,1546318800,64.12,51.19,0.0061,0.0405,1.546326e+09,rain,7.29,1546318800,1.77,310,0.77,2.49,2019-01-01 05:00:00,ATL,61580
1,33.6558,-84.4333,1546405200,57.07,49.10,0.0146,0.0764,1.546481e+09,rain,7.33,1546452000,1.29,28,0.84,2.02,2019-01-02 05:00:00,ATL,61580
2,33.6558,-84.4333,1546491600,54.52,53.34,0.0118,0.0798,1.546495e+09,rain,8.03,1546538400,1.26,7,1.00,1.89,2019-01-03 05:00:00,ATL,61580
3,33.6558,-84.4333,1546578000,64.76,43.52,0.0476,0.2179,1.546600e+09,rain,20.46,1546639200,1.54,201,0.91,2.68,2019-01-04 05:00:00,ATL,61580
4,33.6558,-84.4333,1546664400,58.52,41.86,0.0000,0.0000,NaN,NaN,15.32,1546714800,5.80,291,0.41,3.23,2019-01-05 05:00:00,ATL,61580


In [17]:
wa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 640 entries, 0 to 639
Data columns (total 18 columns):
latitude                    640 non-null float64
longitude                   640 non-null float64
time                        640 non-null int64
temperature_high            640 non-null float64
temperature_low             640 non-null float64
precip_intensity            640 non-null float64
precip_intensity_max        640 non-null float64
precip_intensity_maxtime    193 non-null float64
precip_type                 166 non-null object
wind_gust                   640 non-null float64
wind_gust_time              640 non-null int64
wind_speed                  640 non-null float64
wind_bearing                640 non-null int64
cloud_cover                 640 non-null float64
visibility                  640 non-null float64
datetime                    640 non-null datetime64[ns]
airports                    640 non-null object
count                       640 non-null int64
dtypes: datetime

In [18]:
wa.drop(['count','time','precip_intensity_maxtime', 'wind_gust_time'],axis=1, inplace=True)


In [19]:
wa['datetime'] = pd.to_datetime(wa['datetime'], format='%Y-%m-%d')

In [20]:
wa['date'] = [d.date() for d in wa['datetime']]
wa['date'] = pd.to_datetime(wa['date'], format='%Y-%m-%d')

In [21]:
wa['year'] = pd.DatetimeIndex(wa['date']).year
wa['month'] = pd.DatetimeIndex(wa['date']).month
wa['day_of_month'] = pd.DatetimeIndex(wa['date']).day

In [22]:
wa['precip_type'].fillna('None', inplace=True)

In [23]:
wa.fillna(0, inplace=True)

In [24]:
wa.head()

,latitude,longitude,temperature_high,temperature_low,precip_intensity,precip_intensity_max,precip_type,wind_gust,wind_speed,wind_bearing,cloud_cover,visibility,datetime,airports,date,year,month,day_of_month
0,33.6558,-84.4333,64.12,51.19,0.0061,0.0405,rain,7.29,1.77,310,0.77,2.49,2019-01-01 05:00:00,ATL,2019-01-01,2019,1,1
1,33.6558,-84.4333,57.07,49.10,0.0146,0.0764,rain,7.33,1.29,28,0.84,2.02,2019-01-02 05:00:00,ATL,2019-01-02,2019,1,2
2,33.6558,-84.4333,54.52,53.34,0.0118,0.0798,rain,8.03,1.26,7,1.00,1.89,2019-01-03 05:00:00,ATL,2019-01-03,2019,1,3
3,33.6558,-84.4333,64.76,43.52,0.0476,0.2179,rain,20.46,1.54,201,0.91,2.68,2019-01-04 05:00:00,ATL,2019-01-04,2019,1,4
4,33.6558,-84.4333,58.52,41.86,0.0000,0.0000,None,15.32,5.80,291,0.41,3.23,2019-01-05 05:00:00,ATL,2019-01-05,2019,1,5


In [25]:
wa_cols = wa.columns.tolist()
wa_cols

['latitude',
 'longitude',
 'temperature_high',
 'temperature_low',
 'precip_intensity',
 'precip_intensity_max',
 'precip_type',
 'wind_gust',
 'wind_speed',
 'wind_bearing',
 'cloud_cover',
 'visibility',
 'datetime',
 'airports',
 'date',
 'year',
 'month',
 'day_of_month']

In [26]:
wa_cols.insert(0, wa_cols.pop(wa_cols.index('year')))
wa_cols.insert(1, wa_cols.pop(wa_cols.index('month')))
wa_cols.insert(2, wa_cols.pop(wa_cols.index('day_of_month')))
wa = wa.reindex(columns=wa_cols)
wa.head()

,year,month,day_of_month,latitude,longitude,temperature_high,temperature_low,precip_intensity,precip_intensity_max,precip_type,wind_gust,wind_speed,wind_bearing,cloud_cover,visibility,datetime,airports,date
0,2019,1,1,33.6558,-84.4333,64.12,51.19,0.0061,0.0405,rain,7.29,1.77,310,0.77,2.49,2019-01-01 05:00:00,ATL,2019-01-01
1,2019,1,2,33.6558,-84.4333,57.07,49.10,0.0146,0.0764,rain,7.33,1.29,28,0.84,2.02,2019-01-02 05:00:00,ATL,2019-01-02
2,2019,1,3,33.6558,-84.4333,54.52,53.34,0.0118,0.0798,rain,8.03,1.26,7,1.00,1.89,2019-01-03 05:00:00,ATL,2019-01-03
3,2019,1,4,33.6558,-84.4333,64.76,43.52,0.0476,0.2179,rain,20.46,1.54,201,0.91,2.68,2019-01-04 05:00:00,ATL,2019-01-04
4,2019,1,5,33.6558,-84.4333,58.52,41.86,0.0000,0.0000,None,15.32,5.80,291,0.41,3.23,2019-01-05 05:00:00,ATL,2019-01-05


In [64]:
# bad_date = wa[wa['day_of_month']==31].index
# wa.drop(bad_date, inplace=True)

## Averages

In [ ]:
avg_dep = wa.groupby('airports').mean()
avg_dep.reset_index(drop=False,inplace=True)
avg_dep.head()

In [ ]:
avg_dep.rename(index=str, columns={'airports': 'origin'
                                   , 'temperature_high':'avg_temp_high'
                                   ,'temperature_low':'avg_temp_low'
                                   , 'precip_intensity':'avg_precip'
                                   ,'precip_intensity_max':'avg_max_precip'
                                   ,'wind_gust':'avg_wind_gust'
                                   ,'visibility':'avg_dep_visibility'
                                   ,'wind_speed':'avg_wind_speed'
                                   ,'wind_bearing': 'avg_wind_bearing'
                                   ,'cloud_cover':'avg_cloud_cover'},inplace=True)

In [ ]:
avg_dep.shape

In [ ]:
avg_arr = wa.groupby('airports').mean()
avg_arr.reset_index(drop=False,inplace=True)

avg_arr.rename(index=str, columns={'airports': 'dest'
                                   , 'temperature_high':'avg_temp_high'
                                   ,'temperature_low':'avg_temp_low'
                                   , 'precip_intensity':'avg_precip'
                                   ,'precip_intensity_max':'avg_max_precip'
                                   ,'wind_gust':'avg_wind_gust'
                                   ,'visibility':'avg_dep_visibility'
                                   ,'wind_speed':'avg_wind_speed'
                                   ,'wind_bearing': 'avg_wind_bearing'
                                   ,'cloud_cover':'avg_cloud_cover'},inplace=True)

In [ ]:
avg_arr

## Origin and Destination Weather Data

In [27]:
origin_weather_df = wa.copy()
dest_weather_df = wa.copy()

origin_weather_df.rename(index=str, columns={'airports':'origin'
                                             ,'temperature_high':'origin_temp_high'
                                             ,'temperature_low':'origin_temp_low'
                                             , 'precip_intensity':'origin_precip'
                                             ,'precip_type':'origin_precip_type'
                                             ,'precip_intensity_max':'origin_max_precip'
                                             ,'wind_gust':'origin_wind_gust'
                                             ,'visibility':'origin_dep_visibility'
                                             ,'wind_speed':'origin_wind_speed'
                                             ,'wind_bearing': 'origin_wind_bearing'
                                             ,'cloud_cover':'origin_cloud_cover'
                                             ,'date':'fl_date'},inplace=True)

In [28]:
origin_weather_df.head()

,year,month,day_of_month,latitude,longitude,origin_temp_high,origin_temp_low,origin_precip,origin_max_precip,origin_precip_type,origin_wind_gust,origin_wind_speed,origin_wind_bearing,origin_cloud_cover,origin_dep_visibility,datetime,origin,fl_date
0,2019,1,1,33.6558,-84.4333,64.12,51.19,0.0061,0.0405,rain,7.29,1.77,310,0.77,2.49,2019-01-01 05:00:00,ATL,2019-01-01
1,2019,1,2,33.6558,-84.4333,57.07,49.10,0.0146,0.0764,rain,7.33,1.29,28,0.84,2.02,2019-01-02 05:00:00,ATL,2019-01-02
2,2019,1,3,33.6558,-84.4333,54.52,53.34,0.0118,0.0798,rain,8.03,1.26,7,1.00,1.89,2019-01-03 05:00:00,ATL,2019-01-03
3,2019,1,4,33.6558,-84.4333,64.76,43.52,0.0476,0.2179,rain,20.46,1.54,201,0.91,2.68,2019-01-04 05:00:00,ATL,2019-01-04
4,2019,1,5,33.6558,-84.4333,58.52,41.86,0.0000,0.0000,None,15.32,5.80,291,0.41,3.23,2019-01-05 05:00:00,ATL,2019-01-05


In [29]:
dest_weather_df.rename(index=str, columns={'airports':'dest'
                                             ,'temperature_high':'dest_temp_high'
                                             ,'temperature_low':'dest_temp_low'
                                             , 'precip_intensity':'dest_precip'
                                             ,'precip_type':'dest_precip_type'
                                             ,'precip_intensity_max':'dest_max_precip'
                                             ,'wind_gust':'dest_wind_gust'
                                             ,'visibility':'dest_dep_visibility'
                                             ,'wind_speed':'dest_wind_speed'
                                             ,'wind_bearing': 'dest_wind_bearing'
                                             ,'cloud_cover':'dest_cloud_cover'
                                           ,'date':'fl_date'},inplace=True)

In [30]:
df_sub = pd.merge(df_sub, origin_weather_df, on=['fl_date','origin'], how='left')
df_sub = pd.merge(df_sub, dest_weather_df, on=['fl_date','dest'], how='left')

In [31]:
df_sub.columns

Index(['year_x', 'month_x', 'day_of_week', 'fl_date', 'op_unique_carrier',
       'tail_num', 'origin', 'dest', 'crs_dep_time', 'dep_time', 'dep_delay',
       'dep_delay_new', 'crs_arr_time', 'arr_time', 'arr_delay',
       'arr_delay_new', 'actual_elapsed_time', 'distance', 'dep_hour',
       'arr_hour', 'Code', 'Description', 'prev_arr_delay',
       'avg_prev_arr_delay', 'year_y', 'month_y', 'day_of_month_x',
       'latitude_x', 'longitude_x', 'origin_temp_high', 'origin_temp_low',
       'origin_precip', 'origin_max_precip', 'origin_precip_type',
       'origin_wind_gust', 'origin_wind_speed', 'origin_wind_bearing',
       'origin_cloud_cover', 'origin_dep_visibility', 'datetime_x', 'year',
       'month', 'day_of_month_y', 'latitude_y', 'longitude_y',
       'dest_temp_high', 'dest_temp_low', 'dest_precip', 'dest_max_precip',
       'dest_precip_type', 'dest_wind_gust', 'dest_wind_speed',
       'dest_wind_bearing', 'dest_cloud_cover', 'dest_dep_visibility',
       'datetime_y']

In [32]:
df_sub.drop(['year_x', 'month_x','year_y', 'month_y', 'day_of_month_x','datetime_x'
             ,'day_of_month_y','datetime_y'],axis=1, inplace=True)

In [ ]:
# df_sub = pd.merge(df_sub,avg_dep,how='left',on='origin')
# df_sub = pd.merge(df_sub,avg_arr,how='left',on='dest')

In [31]:
df_sub.to_csv('full_2019_1_data2.csv', index=False)

## Creating Targets and Features

Now that we have all of our data merged properly, I will clean a few things such as redundant values, unnecessary columns, and do some feature engineering such as adding dummy variables. I will also add my target column which will be whether a flight was delayed or not. 

First, I will remove any duplicate hours (00 and 24), both are midnight. Therefore, I will convert these to 0 and then drop the na values. 

In [33]:
# Remove any duplicate hours (aka 00 and 24)
df_sub['arr_hour'] = df_sub['arr_hour'].apply(lambda x:0 if x==24 else x)
df_sub.dropna(inplace=True)
df_sub.shape

(122657, 48)

In [34]:
df_sub.columns

Index(['day_of_week', 'fl_date', 'op_unique_carrier', 'tail_num', 'origin',
       'dest', 'crs_dep_time', 'dep_time', 'dep_delay', 'dep_delay_new',
       'crs_arr_time', 'arr_time', 'arr_delay', 'arr_delay_new',
       'actual_elapsed_time', 'distance', 'dep_hour', 'arr_hour', 'Code',
       'Description', 'prev_arr_delay', 'avg_prev_arr_delay', 'latitude_x',
       'longitude_x', 'origin_temp_high', 'origin_temp_low', 'origin_precip',
       'origin_max_precip', 'origin_precip_type', 'origin_wind_gust',
       'origin_wind_speed', 'origin_wind_bearing', 'origin_cloud_cover',
       'origin_dep_visibility', 'year', 'month', 'latitude_y', 'longitude_y',
       'dest_temp_high', 'dest_temp_low', 'dest_precip', 'dest_max_precip',
       'dest_precip_type', 'dest_wind_gust', 'dest_wind_speed',
       'dest_wind_bearing', 'dest_cloud_cover', 'dest_dep_visibility'],
      dtype='object')

Next, I will drop a few columns I've deemed unnecessary for my modeling.

In [35]:
df_sub.drop(['year','fl_date','dep_delay_new','latitude_x',
       'longitude_x', 'latitude_y', 'longitude_y','Code', 'Description'],axis=1, inplace=True)

I will then convert my categorical variables into dummy variables.

In [36]:
#Convert to Dummy Variables
# df = pd.concat([df,pd.get_dummies(df['month'],drop_first=True,prefix="MONTH")],axis=1)
df_sub = pd.concat([df_sub,pd.get_dummies(df_sub['day_of_week'],drop_first=True,prefix="day_of_week")],axis=1)
df_sub = pd.concat([df_sub,pd.get_dummies(df_sub['op_unique_carrier'],drop_first=True,prefix="op_unique_carrier")],axis=1)
df_sub = pd.concat([df_sub,pd.get_dummies(df_sub['origin'],drop_first=True,prefix="origin")],axis=1)
df_sub = pd.concat([df_sub,pd.get_dummies(df_sub['dest'],drop_first=True,prefix="dest")],axis=1)
df_sub = pd.concat([df_sub,pd.get_dummies(df_sub['dep_hour'],drop_first=True,prefix="dep_hour")],axis=1)
df_sub = pd.concat([df_sub,pd.get_dummies(df_sub['arr_hour'],drop_first=True,prefix="arr_hour")],axis=1)
df_sub = pd.concat([df_sub,pd.get_dummies(df_sub['dest_precip_type'],drop_first=True,prefix="dest_precip_type")],axis=1)
df_sub = pd.concat([df_sub,pd.get_dummies(df_sub['origin_precip_type'],drop_first=True,prefix="origin_precip_type")],axis=1)
# drop these columns
df_sub.drop(['month', 'day_of_week', 'op_unique_carrier', 'dest_precip_type', 'origin_precip_type','origin', 'dest', 'dep_hour','arr_hour'], axis=1, inplace=True)

Now that we've cleaned most of the data up, I will create my target column. To create this, I will take the `arr_delay` column and categorize flights delayed more than 30 minutes as delayed. Fligths that are deemed delayed will be noted as "1" for yes, flights not delayed will be noted as "0" for no.

In [37]:
df_sub['delayed'] = df_sub['arr_delay'].apply(lambda x:1 if x>=30 else 0)

I have my target, let's see how it is dispersed within my dataset.

In [38]:
df_sub['delayed'].value_counts()

0    106153
1     16504
Name: delayed, dtype: int64

In [37]:
df_sub.to_csv('cleaned_target_30min_jan.csv', index=False)

In [83]:
df_sub.head()

,tail_num,crs_dep_time,dep_time,dep_delay,crs_arr_time,arr_time,arr_delay,arr_delay_new,actual_elapsed_time,distance,...,arr_hour_21,arr_hour_22,arr_hour_23,dest_precip_type_rain,dest_precip_type_sleet,dest_precip_type_snow,origin_precip_type_rain,origin_precip_type_sleet,origin_precip_type_snow,delayed
0,N110AN,800,759.0,-1.0,1133,1058.0,-35.0,0.0,359.0,2475.0,...,0,0,0,0,0,0,0,0,0,0
1,N116AN,800,849.0,49.0,1133,1156.0,23.0,23.0,367.0,2475.0,...,0,0,0,1,0,0,0,0,0,0
2,N105NN,800,756.0,-4.0,1133,1108.0,-25.0,0.0,372.0,2475.0,...,0,0,0,1,0,0,0,0,0,0
3,N108NN,800,756.0,-4.0,1133,1126.0,-7.0,0.0,390.0,2475.0,...,0,0,0,1,0,0,0,0,0,0
4,N110AN,800,755.0,-5.0,1133,1112.0,-21.0,0.0,377.0,2475.0,...,0,0,0,1,0,0,0,0,0,0


In [39]:
feb = pd.read_csv('cleaned_target_30min_feb.csv')
feb.head()

,tail_num,crs_dep_time,dep_time,dep_delay,crs_arr_time,arr_time,arr_delay,arr_delay_new,actual_elapsed_time,distance,...,arr_hour_21,arr_hour_22,arr_hour_23,dest_precip_type_rain,dest_precip_type_sleet,dest_precip_type_snow,origin_precip_type_rain,origin_precip_type_sleet,origin_precip_type_snow,delayed
0,N110AN,800,759.0,-1.0,1133,1058.0,-35.0,0.0,359.0,2475.0,...,0,0,0,0,0,0,0,0,0,0
1,N116AN,800,849.0,49.0,1133,1156.0,23.0,23.0,367.0,2475.0,...,0,0,0,1,0,0,0,0,0,0
2,N105NN,800,756.0,-4.0,1133,1108.0,-25.0,0.0,372.0,2475.0,...,0,0,0,1,0,0,0,0,0,0
3,N108NN,800,756.0,-4.0,1133,1126.0,-7.0,0.0,390.0,2475.0,...,0,0,0,1,0,0,0,0,0,0
4,N110AN,800,755.0,-5.0,1133,1112.0,-21.0,0.0,377.0,2475.0,...,0,0,0,1,0,0,0,0,0,0


In [40]:
combined = pd.concat([feb, df_sub], join='inner')
combined.head()

,tail_num,crs_dep_time,dep_time,dep_delay,crs_arr_time,arr_time,arr_delay,arr_delay_new,actual_elapsed_time,distance,...,arr_hour_19,arr_hour_20,arr_hour_21,arr_hour_22,arr_hour_23,dest_precip_type_rain,dest_precip_type_snow,origin_precip_type_rain,origin_precip_type_snow,delayed
0,N110AN,800,759.0,-1.0,1133,1058.0,-35.0,0.0,359.0,2475.0,...,0,0,0,0,0,0,0,0,0,0
1,N116AN,800,849.0,49.0,1133,1156.0,23.0,23.0,367.0,2475.0,...,0,0,0,0,0,1,0,0,0,0
2,N105NN,800,756.0,-4.0,1133,1108.0,-25.0,0.0,372.0,2475.0,...,0,0,0,0,0,1,0,0,0,0
3,N108NN,800,756.0,-4.0,1133,1126.0,-7.0,0.0,390.0,2475.0,...,0,0,0,0,0,1,0,0,0,0
4,N110AN,800,755.0,-5.0,1133,1112.0,-21.0,0.0,377.0,2475.0,...,0,0,0,0,0,1,0,0,0,0


In [40]:
combined.to_csv('full_30_data.csv', index=False)

In [41]:
combined['delayed'].value_counts()

0    199686
1     35457
Name: delayed, dtype: int64